# Configuração e Script para Listar Buckets no Amazon S3

Este guia passo a passo mostra como configurar o AWS CLI usando AWS SSO (Single Sign-On) e como executar um script Python para listar os buckets no Amazon S3.

## Configuração do AWS CLI com AWS SSO

Para usar AWS SSO, você precisa configurar o AWS CLI para autenticação SSO e criar um perfil. Siga os passos abaixo:

### 1. Configurar AWS CLI com AWS SSO

1. **Configurar AWS SSO**:
   ```sh
   aws configure sso

#### Seguir as instruções:
- Insira a URL do portal do SSO.
- Insira a região do SSO.
- Siga o link gerado para se autenticar.
- Selecione a conta e permissões desejadas.
- Dê um nome ao perfil (ex.: default).

**Uma vez criado o usuário, podemos realizar o login sempre que necessário, sem precisar configurar novamente, através do comando:**
   ```sh
   aws sso login --profile default

## Listar Buckets no Amazon S3 usando boto3

Este script Python utiliza a biblioteca boto3 para listar os buckets no Amazon S3. Ele faz uso de perfis configurados na AWS CLI para autenticação.

### Pré-requisitos

1. **Instalar boto3**
   Certifique-se de que boto3 está instalado. Você pode instalá-lo usando pip:
   ```sh
   pip install boto3

### Funcionamento
- Importações: Importa boto3 para interagir com os serviços da AWS e as exceções necessárias do botocore.

- Função list_s3_buckets:
- Sessão e Cliente S3: Inicializa uma sessão usando o perfil AWS configurado e cria um cliente S3.
- Listagem de Buckets: Obtém a lista de buckets chamando s3.list_buckets().
- Verificação e Impressão: Verifica se a lista de buckets está vazia e imprime os nomes dos buckets encontrados.
- Tratamento de Exceções: Lida com exceções relacionadas a credenciais ausentes ou incompletas e outros erros gerais.

In [2]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError


def list_s3_buckets():
    try:
        # Inicializa o cliente S3 usando o perfil configurado
        session = boto3.Session(profile_name="default")
        s3 = session.client("s3")

        # Lista os buckets
        response = s3.list_buckets()

        buckets = response.get("Buckets", [])

        if not buckets:
            print("Nenhum bucket encontrado.")
        else:
            print("Buckets encontrados:")
            for bucket in buckets:
                print(f" - {bucket['Name']}")

    except NoCredentialsError:
        print("Credenciais não encontradas.")
    except PartialCredentialsError:
        print("Credenciais incompletas.")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


if __name__ == "__main__":
    list_s3_buckets()

Buckets encontrados:
 - desafiosprintrodrigo


## Criação de Bucket no Amazon S3

Este guia demonstra como criar um bucket no Amazon S3 usando Python e boto3, garantindo que o nome do bucket seja único ao adicionar um identificador UUID.

### Pré-requisitos

1. **AWS CLI Configurado**: Certifique-se de que você configurou o AWS CLI e autenticou com sucesso usando AWS SSO ou chaves de acesso.
2. **Biblioteca boto3**: Instale boto3 se ainda não tiver instalado. Você pode fazer isso usando pip:
   ```sh
   pip install boto3

#### Explicação do Código

##### Importações:

- boto3 para interagir com os serviços da AWS.

**Função create_s3_bucket:**

- Inicialização do Cliente S3: Cria uma sessão boto3 usando o perfil padrão configurado.

- Criação do Bucket: Verifica a região e cria o bucket. Se a região for us-east-1, não configura LocationConstraint.

- Tratamento de Exceções: Lida com exceções específicas e genéricas, incluindo a verificação de buckets já existentes.

##### Uso da Função:

Chama a função create_s3_bucket com o nome do bucket e a região especificada.

In [1]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError


def create_s3_bucket(bucket_name, region=None):
    try:
        # Inicializa a sessão e o cliente S3 usando o perfil configurado
        session = boto3.Session(profile_name="default")
        s3_client = session.client("s3", region_name=region)

        # Cria o bucket com a região especificada
        if region is None or region == "us-east-1":
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {"LocationConstraint": region}
            s3_client.create_bucket(
                Bucket=bucket_name, CreateBucketConfiguration=location
            )

        print(f"Bucket '{bucket_name}' criado com sucesso.")

    except NoCredentialsError:
        print("Credenciais não encontradas.")
    except PartialCredentialsError:
        print("Credenciais incompletas.")
    except ClientError as e:
        print(f"Ocorreu um erro ao tentar criar o bucket: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


bucket_name = "desafiosprintrodrigo"
region = "us-east-1"
create_s3_bucket(bucket_name, region)

Bucket 'desafiosprintrodrigo' criado com sucesso.


## Upload do Arquivo para o Bucket

### Importações:

- Importa **boto3** para interagir com os serviços da AWS.
- Importa exceções relevantes do **botocore**.

### Função upload_file_to_s3:

- Inicialização do Cliente S3: Cria uma sessão boto3 usando o perfil padrão configurado e inicializa o cliente S3.
- Nome do Objeto: Se object_name não for fornecido, usa o file_path como nome do objeto no S3.
- Upload do Arquivo: Usa s3_client.upload_file para enviar o arquivo para o bucket S3.
- Tratamento de Exceções: Lida com exceções específicas e genéricas.

### Uso da Função:

- Define bucket_name, file_path, e object_name (opcional).
- Chama a função upload_file_to_s3 para fazer o upload do arquivo para o bucket S3.

In [3]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError


def upload_file_to_s3(bucket_name, file_path, object_name=None):
    try:
        # Inicializa a sessão e o cliente S3 usando o perfil configurado
        session = boto3.Session(profile_name="default")
        s3_client = session.client("s3")

        # Se o nome do objeto não for especificado, usa o nome do arquivo
        if object_name is None:
            object_name = file_path

        # Faz o upload do arquivo
        s3_client.upload_file(file_path, bucket_name, object_name)

        print(
            f"Arquivo '{file_path}' enviado para o bucket '{bucket_name}' como '{object_name}'."
        )

    except NoCredentialsError:
        print("Credenciais não encontradas.")
    except PartialCredentialsError:
        print("Credenciais incompletas.")
    except ClientError as e:
        print(f"Ocorreu um erro ao tentar enviar o arquivo: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


bucket_name = "desafiosprintrodrigo"
file_path = "date/infracos_transito_07_2023.csv"
object_name = "infracos_transito_07_2023.csv"

upload_file_to_s3(bucket_name, file_path, object_name)

Arquivo 'date/infracos_transito_07_2023.csv' enviado para o bucket 'desafiosprintrodrigo' como 'infracos_transito_07_2023.csv'.


## Consulta SQL com Pandasql

### Descrição:

Este código Python realiza uma consulta SQL em um DataFrame do Pandas utilizando a biblioteca pandasql. Ele faz o upload de um arquivo CSV de um bucket S3 da AWS, lê esse arquivo em um DataFrame do Pandas e executa uma consulta SQL contida em um arquivo externo.

#### Função openQuery:

- Descrição: Esta função é responsável por abrir um arquivo contendo uma consulta SQL e retornar o conteúdo como uma string.
- Parâmetros de entrada: Recebe o nome do arquivo contendo a consulta SQL.
- Saída: Retorna a consulta SQL como uma string.

#### Uso da Função:
- Bucket S3: Define o nome do bucket S3 de onde será feito o download do arquivo CSV.
- Arquivo CSV: Define o nome do arquivo CSV no bucket S3 a ser baixado.
- Consulta SQL: Define o nome do arquivo contendo a consulta SQL a ser executada.
- Leitura do Arquivo CSV: Utiliza a biblioteca boto3 para fazer o download do arquivo CSV do bucket S3 e a função read_csv do Pandas para ler o arquivo em um DataFrame do Pandas.
- Execução da Consulta SQL: Utiliza o pandasql para executar a consulta SQL no DataFrame do Pandas.
- Exportação do Resultado: Exporta o resultado da consulta para um arquivo CSV localmente.

#### Dependências:

- boto3: Utilizada para interagir com os serviços da AWS, neste caso, para baixar o arquivo CSV do S3.
- pandas: Utilizada para manipulação e análise de dados, incluindo a leitura do arquivo CSV em um DataFrame.
- pandasql: Utilizada para executar consultas SQL em DataFrames do Pandas.
- botocore.exceptions: Importa exceções relevantes do botocore, que podem ser tratadas durante a interação com o cliente S3.

##### Exceções Tratadas:
Erro ao Ler Arquivo CSV: Trata exceções que podem ocorrer ao tentar ler o arquivo CSV do S3.

In [4]:
import boto3
import pandas as pd
import pandasql as ps
from botocore.exceptions import ClientError

session = boto3.Session(profile_name='default')
sess = boto3.Session(region_name='us-east-1')
s3Client = session.client('s3')

bucket_name = 'desafiosprintrodrigo'

def openQuery(nomeQuery):
    with open(nomeQuery, 'r') as qy:
        query = qy.read() 
        return query


try:
    response = s3Client.get_object(Bucket=bucket_name, Key='infracos_transito_07_2023.csv')
    df = pd.read_csv(response['Body'], delimiter=';')
    consulta = openQuery('consulta.sql')
    print(ps.sqldf(consulta, locals()))
    
    dataframe_como_csv = pd.DataFrame(ps.sqldf(consulta, locals()))
    dataframe_como_csv.to_csv(r'consulta.csv', index=False)
   
    
except Exception as e:
    print(f"Erro ao ler o arquivo CSV do S3: {e}")

   quantidade_registros               tipo_veiculo  infracoes_velocidade  \
0                 25379  AUTOMOVEL                                 20046   
1                 18234                  AUTOMOVEL                 16549   

   quantidade_condutores latitude_media  ano_cometimento  \
0                  25379           None             2023   
1                  18234           None             2023   

                                     descricao_curta  data_atual  
0  Transitar em velocidade superior à máxima perm...  2024-05-20  
1  Transitar em velocidade superior à máxima perm...  2024-05-20  
